This notebook will create the config file and the ner model for the food
we will add this model pipeline with existing spacy pipeline in order to make a final model which should theoretically be able to identify
the food items and the amounts correctly

python -m spacy init fill-config base_config.cfg config.cfg

python -m spacy train SpaCy/TRAIN/configGPU.cfg --output SpaCy/model/  --paths.train datasets/spacyFiles/TrainData.spacy --paths.dev datasets/spacyFIles/TrainData.spacy --gpu-id 0


python -m spacy evaluate [path to model] [test.spacy path]

In [41]:
import spacy
from spacy import displacy
spacy.prefer_gpu()
nlp = spacy.load("../model/combined/")

In [42]:

# nlp = spacy.load("en_core_web_trf",disable=["tagger","parser", "attribute_ruler", "lemmatizer"])
# food_nlp = spacy.load("C:\\Users\\samad\\Documents\\DISSERTATION\\SpaCy\\model\\model-best")
# food_nlp.replace_listeners("transformer", "ner", ["model.tok2vec"])

# nlp.add_pipe(
#     "ner",
#     name="ner_food",
#     source=food_nlp,
#     after="ner",
# )

# # nlp.to_disk("../model/combined/")


    cant change the token in a span so we just gonna convert it and save to array
    so a cardinal if its like "THREE" use numerizer() to get numerical val and use that instead

    so that the ents list is only cardinal and FOOD
    and say 
    if the item that we picked is a cardinal
        if the next ents is food then we use numerize on the cardinal
         [FOOD, cardinal._.numerize()]
    if the item picked is FOOD and next is also FOOD
        [FOOD,1]

    #if we are in a number and the next is food
    #logically thinking the number is quantity for the food 
    # while this is usual case, ofc this code can be rendered useless easily
    # because someone can say "i ate chicken wings, 20 of them"
    # and now this code will not work 😕
    
    #maybe skipping the next ent if the current is cardinal
    # eg cardinal food food food cardinal food
    # i = 0 we are at cardinal and next is food so we save [food,numerizedcardinal], then i+= 1 and also the i+=1 at the end of the while funct
    # i = 2 its food  so just i+=1
    # i = 3 its food again so just i+=1
    # i = 4 its cardinal and next food 

In [51]:

def createList(doc,arr):
    i = 0 
    lenEnts = len(doc.ents)
    while i < lenEnts:
        currEnt = doc.ents[i] # the current ent 
        print(i , currEnt.label_)
        try: 
            nextEnt = doc.ents[i+1] # the next ent in order to check
            print(i,nextEnt.label_)
            if currEnt.label_ == "CARDINAL" and nextEnt.label_ == "FOOD":
                i+=1
                arr.append([nextEnt.text,currEnt._.numerize()])
            else:
                if (currEnt.label_ == "FOOD"): 
                    arr.append([currEnt.text,1])
        except: 
            # print("no next ent at"+str(i))
            if(currEnt.label_ == "FOOD"):
                arr.append([currEnt.text,1])
            pass
        i+=1
    return arr

In [52]:
from spacy import load
import numerizer

# doc = nlp("I ate three cereal bar, half of an apple and a banana for breakfast and a ham sandwich for lunch and chicken and rice for dinner and 3 chicken thighs and chicken")

doc = nlp("800 grams of Locatelli's  ribeye steak")

displacy.render(doc,style="ent",jupyter=True)
arr = []

#if the ent list is larger than 1 then we can do the 
if len(doc.ents) == 1 and doc.ents[0].label_ == 'FOOD':
    arr.append([doc.ents[0].text , 1])
else:
    arr = createList(doc,arr)


print(arr)
print(doc.ents)



0 QUANTITY
0 ORG
1 ORG
1 FOOD
2 FOOD
[['ribeye steak', 1]]
(800 grams, Locatelli, ribeye steak)


In [63]:
from itertools import *

doc = nlp("I ate three cereal bar, half of an apple and 500 g of ribeye steak and a ham sandwich for lunch and chicken and rice for dinner and 3 chicken thighs and chicken")
# e = {key:list(g) for key, g in groupby(doc.ents,lambda x:x.label_)}

displacy.render(doc,style="ent",jupyter=True)
entities = {key: list(g) for key, g in groupby(sorted(doc.ents, key=lambda x: x.label_), lambda x: x.label_)} 

print(entities)

{'CARDINAL': [three, half, 3], 'FOOD': [cereal bar, apple, ribeye steak, ham sandwich, chicken and rice, chicken thighs, chicken], 'QUANTITY': [500 g]}
